In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
import category_encoders as ce

In [21]:
import dagshub
dagshub.init(repo_owner='sourav664', repo_name='real-estate-price-prediction', mlflow=True)


Accessing as sourav664

Initialized MLflow to track repo "sourav664/real-estate-price-prediction"

Repository sourav664/real-estate-price-prediction initialized!

In [22]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/sourav664/real-estate-price-prediction.mlflow")

In [23]:
# mlflow experiment

mlflow.set_experiment("Exp 2 - Random Forest Hyperparameter Tuning")

<Experiment: artifact_location='mlflow-artifacts:/4f3b4bf571034811aac969d2a9746e2e', creation_time=1744435694035, experiment_id='1', last_update_time=1744435694035, lifecycle_stage='active', name='Exp 2 - Random Forest Hyperparameter Tuning', tags={}>

In [5]:

from sklearn import set_config

set_config(transform_output="pandas")

In [106]:
df = pd.read_csv("../data/raw/real_estatesv9.csv")

In [107]:
df = df[['price','carpet_area','bedroom','bathroom','transaction_type','floor_category','luxury_category','property_type','regions','balconies_iter','furnished_status_imputed','additional_room']]

In [108]:
df.duplicated().sum(
    
)

286

In [109]:
df.drop_duplicates(inplace=True)

In [88]:
df.shape

(17820, 12)

In [89]:
df['price'].describe()

count    17820.000000
mean         2.117472
std          2.253744
min          0.250000
25%          0.800000
50%          1.400000
75%          2.500000
max         20.000000
Name: price, dtype: float64

In [110]:
X = df.drop(columns=["price"])
y = df["price"]

In [111]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
     

In [112]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (14305, 11)
The shape of test data is (3577, 11)


In [113]:
ohe_encode = ['transaction_type']
ordinal_encode = ['luxury_category','floor_category']
target_encode = ['regions','property_type']
std_encode = ['bedroom','bathroom','balconies_iter']
robust_encode = ['carpet_area']

In [114]:
luxury_order = ['Low', 'Medium', 'High']

floor_order = ['Low Floor', 'Mid Floor', 'High Floor']

In [115]:
from sklearn.preprocessing import FunctionTransformer

pt = FunctionTransformer(np.log1p, inverse_func=np.expm1, validate=True)
y_train_pt = pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt = pt.transform(y_test.values.reshape(-1,1))

In [116]:
from sklearn.preprocessing import RobustScaler

In [117]:
preprocessor = ColumnTransformer(
    transformers=[
        ("ohe", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False), ohe_encode),
        ("ordinal", OrdinalEncoder(categories=[luxury_order, floor_order]), ordinal_encode),
        
        ("num", StandardScaler(), std_encode),
        ("robust", RobustScaler(), robust_encode)
    ], remainder="passthrough", n_jobs=-1, force_int_remainder_cols=False,verbose_feature_names_out=False
)

preprocessor.set_output(transform="pandas")

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['transaction_type']),
                                ('ordinal',
                                 OrdinalEncoder(categories=[['Low', 'Medium',
                                                             'High'],
                                                            ['Low Floor',
                                                             'Mid Floor',
                                                             'High Floor']]),
                                 ['luxury_category', 'floor_category']),
                                ('num', StandardScaler(),
                                 ['bedroom', 'bathroom', 'balconies_iter']),
                                ('robust', RobustScaler(), ['carpet_area'])],
                  verbose_feature_names_out=False)

In [118]:
# Creating a pipeline
pipeline = Pipeline([
    ('target_encoder', ce.TargetEncoder(cols=target_encode)),
    ('preprocessor', preprocessor)
    
])

In [119]:
# do data preprocessing

X_train_trans = pipeline.fit_transform(X_train, y_train)

X_test_trans = pipeline.transform(X_test)

In [54]:
from sklearn.ensemble import RandomForestRegressor
import optuna

In [55]:
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor

In [56]:
def objective(trial):
    with mlflow.start_run(nested=True):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 1, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            "max_features": trial.suggest_categorical("max_features",[None,"sqrt","log2"]),
            "max_samples": trial.suggest_float("max_samples",0.5,1),
            "random_state": 42,
            "n_jobs": -1,
        }
        
        # log model parameters
        mlflow.log_params(params)
        
        # build the model
        rf = RandomForestRegressor(**params)
        model = TransformedTargetRegressor(regressor=rf, transformer=pt)
        
        # train the model
        model.fit(X_train_trans, y_train)
        
        # get the predictions
        y_pred_train = model.predict(X_train_trans)
        y_pred_test = model.predict(X_test_trans)
        
        # perform cross validation
        cv_scores = cross_val_score(
            model,
            X_train_trans,
            y_train,
            cv=5,
            scoring="neg_mean_absolute_error",
            n_jobs=-1,
        )
        
        # mean score
        mean_score = -(cv_scores.mean())
        
        # log avg cross val error
        mlflow.log_metric("cross_val_error", mean_score)
        
        return mean_score

In [57]:
# create optuna study
study = optuna.create_study(direction='minimize')

with mlflow.start_run(run_name='best_model'):
    # optimize the objective function
    study.optimize(objective, n_trials=20, n_jobs=-1, show_progress_bar=True)
    
    # log the best parameters
    mlflow.log_params(study.best_params)
    
    # log the best score
    mlflow.log_metric("best_score", study.best_value)
    
    
    # train the model on best parameters
    best_rf = RandomForestRegressor(**study.best_params)
    best_rf.fit(X_train_trans, y_train_pt)
    
    # get the predictions
    y_pred_train = best_rf.predict(X_train_trans)
    y_pred_test = best_rf.predict(X_test_trans)

    # get the actual predictions values
    y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
    y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))


    # perform cross validation
    model = TransformedTargetRegressor(regressor=best_rf,
                                        transformer=pt)


    scores = cross_val_score(model,
                         X_train_trans,
                         y_train,
                         scoring="neg_mean_absolute_error",
                         cv=5,n_jobs=-1)

    # log metrics
    mlflow.log_metric("training_error",mean_absolute_error(y_train,y_pred_train_org))
    mlflow.log_metric("test_error",mean_absolute_error(y_test,y_pred_test_org))
    mlflow.log_metric("training_r2",r2_score(y_train,y_pred_train_org))
    mlflow.log_metric("test_r2",r2_score(y_test,y_pred_test_org))
    mlflow.log_metric("cross_val",- scores.mean())

    # log the best model
    mlflow.sklearn.log_model(best_rf,artifact_path="model")

[I 2025-04-17 10:52:46,385] A new study created in memory with name: no-name-e7e1b47b-cccd-4627-80fd-412db5402df9


  0%|          | 0/20 [00:00<?, ?it/s]

🏃 View run treasured-cow-651 at: https://dagshub.com/sourav664/real-estate-price-prediction.mlflow/#/experiments/1/runs/09fbe4aefd3d40ea8dda9917c679d42f
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-price-prediction.mlflow/#/experiments/1
[I 2025-04-17 10:53:05,997] Trial 7 finished with value: 0.701611970556007 and parameters: {'n_estimators': 205, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_samples': 0.7313940788019333}. Best is trial 7 with value: 0.701611970556007.
🏃 View run orderly-quail-966 at: https://dagshub.com/sourav664/real-estate-price-prediction.mlflow/#/experiments/1/runs/191085b855294e909bb1a0184ae9b6da
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-price-prediction.mlflow/#/experiments/1
[I 2025-04-17 10:53:12,609] Trial 6 finished with value: 0.6767555996601958 and parameters: {'n_estimators': 164, 'max_depth': 13, 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_features': None, '

c:\Users\soura\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
2025/04/17 10:57:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run best_model at: https://dagshub.com/sourav664/real-estate-price-prediction.mlflow/#/experiments/1/runs/2bc4d85e4ad8419399da160bcd8ddc41
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-price-prediction.mlflow/#/experiments/1


In [58]:
study.best_params

{'n_estimators': 244,
 'max_depth': 21,
 'min_samples_split': 7,
 'min_samples_leaf': 2,
 'max_features': None,
 'max_samples': 0.6790446161357568}

In [59]:
study.best_value

0.6645231224742064

In [52]:
# optimization history plot

optuna.visualization.plot_optimization_history(study)

In [53]:
# plot hyperparameter importance plot

optuna.visualization.plot_param_importances(study)

In [54]:
# slice plot

optuna.visualization.plot_slice(study)

In [55]:
# dataframe of results

study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_max_depth,params_max_features,params_max_samples,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.744999,2025-04-12 11:37:13.318967,2025-04-12 11:37:33.283753,0 days 00:00:19.964786,19,sqrt,0.933035,10,2,364,COMPLETE
1,1,0.742391,2025-04-12 11:37:13.318967,2025-04-12 11:37:37.037137,0 days 00:00:23.718170,23,log2,0.615515,7,10,230,COMPLETE
2,2,0.690650,2025-04-12 11:37:13.318967,2025-04-12 11:37:44.295685,0 days 00:00:30.976718,23,None,0.738763,3,7,290,COMPLETE
3,3,0.754390,2025-04-12 11:37:13.324558,2025-04-12 11:37:53.624457,0 days 00:00:40.299899,15,sqrt,0.643589,10,5,330,COMPLETE
4,4,0.740994,2025-04-12 11:37:13.326542,2025-04-12 11:37:45.306957,0 days 00:00:31.980415,29,sqrt,0.946794,9,10,118,COMPLETE
5,5,0.720057,2025-04-12 11:37:13.328105,2025-04-12 11:37:24.735974,0 days 00:00:11.407869,30,sqrt,0.960316,5,10,133,COMPLETE
6,6,0.689484,2025-04-12 11:37:13.329259,2025-04-12 11:37:57.911351,0 days 00:00:44.582092,29,None,0.503360,2,3,345,COMPLETE
7,7,0.746645,2025-04-12 11:37:13.329259,2025-04-12 11:37:29.701332,0 days 00:00:16.372073,21,log2,0.873472,10,3,269,COMPLETE
8,8,1.027049,2025-04-12 11:37:24.743576,2025-04-12 11:38:02.271427,0 days 00:00:37.527851,2,sqrt,0.822957,2,8,337,COMPLETE
9,9,0.706478,2025-04-12 11:37:29.711047,2025-04-12 11:38:10.672479,0 days 00:00:40.961432,12,None,0.978609,10,5,319,COMPLETE


In [60]:
rf_params = {'n_estimators': 469,
 'max_depth': 22,
 'min_samples_split': 4,
 'min_samples_leaf': 4,
 'max_features': None,
 'max_samples': 0.891044850651133}

In [120]:
rf_params1 = {'n_estimators': 244,
 'max_depth': 21,
 'min_samples_split': 7,
 'min_samples_leaf': 2,
 'max_features': None,
 'max_samples': 0.6790446161357568}

In [121]:
rf = RandomForestRegressor(**rf_params1)
rf.fit(X_train_trans, y_train_pt)

c:\Users\soura\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=21, max_features=None,
                      max_samples=0.6790446161357568, min_samples_leaf=2,
                      min_samples_split=7, n_estimators=244)

In [122]:
# get the predictions
y_pred_train = rf.predict(X_train_trans)
y_pred_test = rf.predict(X_test_trans)

In [123]:
# get the actual predictions values

y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

In [124]:
print(f"The train error is {mean_absolute_error(y_train,y_pred_train_org):.2f} cr")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test_org):.2f} cr")

The train error is 0.54 cr
The test error is 0.73 cr


In [125]:
print(f"The train r2 score is {r2_score(y_train,y_pred_train_org):.2f}")
print(f"The test r2 score is {r2_score(y_test,y_pred_test_org):.2f}")

The train r2 score is 0.74
The test r2 score is 0.69
